### BreakOut Room Study Case

Dalam Study case kali ini, data yang digunakan adalah data transportation_sales.db yang merupakan data tentang penjualan beberapa jenis kendaraan di beberapa cabang toko di berbagai negara.

####Gdrive Connection

In [1]:
# Menghubungkan Google Colab dengan Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Mengimport Packages Library yang diperlukan
import os
import sqlite3 as sql
import pandas as pd

In [3]:
# Mendifinisikan file disimpan dimana
path = "gdrive/MyDrive/Folder Saya/Narasio Data - Data Analyst/Week 4 - SQL for Data Analysis/Dataset - SQL for Data Analysis/"

In [4]:
# Mengecek file
os.listdir(path)

['rexon_metals.db',
 'weather_stations.db',
 'transportation_sales.db',
 'transportasion_sales.db']

In [5]:
#  Membuat connection ke database yang udah ada
conn = sql.connect(path + 'transportation_sales.db')

# Membuat cursor object untuk memanggil SQL statement
cur = conn.cursor()

In [6]:
# Mengecek terdapat tabel apa saja di transportation_sales.db
cur.execute("""
              SELECT 
                  name 
              FROM sqlite_master 
              WHERE type='table'
            """)

print(cur.fetchall())

[('Customers',), ('Products',), ('Orders',), ('mytable',)]


#### Case 1

Pemilik toko mengalami kesulitan saat melihat tabel Orders karena informasi customer dan product hanya berupa ID, sedangkan pemilik toko ingin melihat nama customer beserta kode dan tipe kendaraan yang dibeli

In [7]:
# Melihat Nama Customer dan Tipe Kendaraan yang dibeli
pd.read_sql_query("""
                  SELECT
                    Orders.CUSTOMER_ID,
                    Customers.CUSTOMERNAME,
                    Orders.PRODUCT_ID,
                    Products.PRODUCTLINE
                  FROM Orders
                  LEFT JOIN Customers ON Orders.CUSTOMER_ID = Customers.Customer_ID
                  INNER JOIN Products ON Orders.PRODUCT_ID = Products.PRODUCT_ID
                  """, conn)

,CUSTOMER_ID,CUSTOMERNAME,PRODUCT_ID,PRODUCTLINE
0,3,Lyon Souveniers,2,Classic Cars
1,1,Land of Toys Inc.,4,Motorcycles
2,4,Toys4GrownUps.com,9,Vintage Cars
3,2,Reims Collectables,10,Trucks and Buses
4,7,Daedalus Designs Imports,5,Classic Cars
5,8,Herkku Gifts,11,Vintage Cars
6,2,Reims Collectables,10,Trucks and Buses
7,9,Mini Wheels Co.,8,Classic Cars
8,5,Corporate Gift Ideas Co.,14,Vintage Cars
9,7,Daedalus Designs Imports,12,Trucks and Buses


#### Case 2

Pemilik toko ingin mengetahui apakah terjadi peningkatan atau bahkan penurunan baik jumlah customer maupun jumlah transaksi pembelian di setiap tahunnya, selain itu ingin mengetahui apakah terjadi New Purchase (Pembelian Baru) dari New Customer. Untuk mengetahui hal tersebut, pertama-tama kita harus mendapatkan berapa jumlah uniq customer di setiap tahunnya.

In [36]:
# Ingin mengetahui apakah terjadi peningkatan atau penurunan customer
pd.read_sql_query("""
                  SELECT
                    COUNT(Customers.CUSTOMERNAME) AS Total_Customer,
                    SUM(Orders.QUANTITYORDERED) AS Total_Pembelian,
                    STRFTIME('%Y', Orders.ORDERDATE) AS Tahun
                  FROM
                    Customers
                  INNER JOIN
                    Orders
                  ON Customers.CUSTOMER_ID = Orders.CUSTOMER_ID
                  GROUP BY
                    Tahun
                  """, conn)

,Total_Customer,Total_Pembelian,Tahun
0,13,463,2003
1,13,481,2004
2,4,130,2005


#### Case 3

Sebutkan top 3 product pada tahun 2003. Apakah dari produk-produk tersebut ada yang menjadi favorit dalam 3 tahun berturut-turut?

In [46]:
# Top 3 Product Pada Tahun 2003
pd.read_sql_query("""
                  SELECT
                    Orders.QUANTITYORDERED,
                    STRFTIME('%Y', Orders.ORDERDATE) AS Tahun,
                    Products.PRODUCTCODE,
                    Products.PRODUCTLINE
                  FROM
                    Orders
                  INNER JOIN
                    Products ON Orders.PRODUCT_ID = Products.PRODUCT_ID
                  WHERE
                    Tahun LIKE '2003%'
                  ORDER BY
                    QUANTITYORDERED DESC
                  LIMIT 3
                  """, conn)

,QUANTITYORDERED,Tahun,PRODUCTCODE,PRODUCTLINE
0,49,2003,S12_3148,Classic Cars
1,48,2003,S18_2238,Classic Cars
2,45,2003,S18_2319,Trucks and Buses


In [45]:
# Top 3 Product Pada Tahun 2004
pd.read_sql_query("""
                  SELECT
                    Orders.QUANTITYORDERED,
                    STRFTIME('%Y', Orders.ORDERDATE) AS Tahun,
                    Products.PRODUCTCODE,
                    Products.PRODUCTLINE
                  FROM
                    Orders
                  INNER JOIN
                    Products ON Orders.PRODUCT_ID = Products.PRODUCT_ID
                  WHERE
                    Tahun LIKE '2004%'
                  ORDER BY
                    QUANTITYORDERED DESC
                  LIMIT 3
                  """, conn)

,QUANTITYORDERED,Tahun,PRODUCTCODE,PRODUCTLINE
0,66,2004,S18_2949,Vintage Cars
1,46,2004,S18_2949,Vintage Cars
2,45,2004,S32_1268,Trucks and Buses


In [47]:
# Top 3 Product Pada Tahun 2005
pd.read_sql_query("""
                  SELECT
                    Orders.QUANTITYORDERED,
                    STRFTIME('%Y', Orders.ORDERDATE) AS Tahun,
                    Products.PRODUCTCODE,
                    Products.PRODUCTLINE
                  FROM
                    Orders
                  INNER JOIN
                    Products ON Orders.PRODUCT_ID = Products.PRODUCT_ID
                  WHERE
                    Tahun LIKE '2005%'
                  ORDER BY
                    QUANTITYORDERED DESC
                  LIMIT 3
                  """, conn)

,QUANTITYORDERED,Tahun,PRODUCTCODE,PRODUCTLINE
0,38,2005,S32_1268,Trucks and Buses
1,37,2005,S18_2949,Vintage Cars
2,29,2005,S18_2238,Classic Cars


#### Case 4

Dapatkan data total penjualan dengan status order dalam proses pengiriman (shipped), ditahan (on hold), dan dibatalkan (cancelled) untuk setiap customer.

In [50]:
# Total Penjualan dengan Status Order Tiap Customer
pd.read_sql_query("""
                  SELECT
                    Orders.ORDER_ID,
                    Orders.CUSTOMER_ID,
                    SUM (Orders.SALES) AS Total_Penjualan,
                    STRFTIME('%Y', Orders.ORDERDATE) AS Tahun,
                    Orders.STATUS,
                    Customers.CUSTOMERNAME
                  FROM
                    Orders
                  INNER JOIN
                    Customers
                  ON
                    Orders.CUSTOMER_ID = Customers.Customer_ID
                  GROUP BY
                    Customers.CUSTOMERNAME
                  """, conn)

,ORDER_ID,CUSTOMER_ID,Total_Penjualan,Tahun,STATUS,CUSTOMERNAME
0,10237,10,10071.05,2003,Shipped,Auto Canal Petit
1,10211,5,17442.12,2003,Shipped,Corporate Gift Ideas Co.
2,10159,7,13528.97,2003,Shipped,Daedalus Designs Imports
3,10168,8,10481.09,2003,Cancelled,Herkku Gifts
4,10121,1,13328.64,2003,Shipped,Land of Toys Inc.
5,10107,3,5468.39,2003,On Hold,Lyon Souveniers
6,10188,9,14479.40,2003,Shipped,Mini Wheels Co.
7,10145,2,17849.72,2003,Shipped,Reims Collectables
8,10263,6,15455.61,2004,Shipped,Technics Stores Inc.
9,10134,4,6318.90,2003,Cancelled,Toys4GrownUps.com


#### Case 5

Berapa jumlah order quantity yang terjual masing-masing produk pada tahun 2004?

In [109]:
# Total Jumlah Order Quantity masing-masing Product pada tahun 2004
pd.read_sql_query("""
                  SELECT
                    Products.PRODUCTLINE,
                    Products.PRODUCTCODE,
                    SUM(Orders.QUANTITYORDERED) AS Total_Order,
                    STRFTIME('%Y', Orders.ORDERDATE) AS Tahun,
                    Orders.STATUS
                  FROM
                    Orders
                  LEFT JOIN
                    Products
                  ON Orders.PRODUCT_ID = Products.PRODUCT_ID
                  GROUP BY
                    2
                  HAVING
                    Tahun = '2004'
                  ORDER BY
                    Total_Order DESC
                  """, conn)

,PRODUCTLINE,PRODUCTCODE,Total_Order,Tahun,STATUS
0,Trucks and Buses,S32_1268,124,2004,On Hold
1,Vintage Cars,S24_3151,42,2004,On Hold
2,Classic Cars,S18_4933,42,2004,On Hold
3,Vintage Cars,S18_1342,36,2004,On Hold
4,Classic Cars,S24_3191,23,2004,Shipped
5,Classic Cars,S18_1129,21,2004,Shipped
6,Motorcycles,S10_1678,20,2004,Shipped


Pemilik toko ingin mengetahui hasil penjualan di setiap tahunnya. Oleh karena itu dari tabel Orders, berapakah pendapatan di setiap tahun?

In [111]:
# Total Penjualan tiap tahun
pd.read_sql_query("""
                  SELECT
                    SUM(SALES) AS Total_Penjualan,
                    STRFTIME('%Y', ORDERDATE) AS Tahun
                  FROM
                    Orders
                  GROUP BY
                    Tahun
                  """, conn)

,Total_Penjualan,Tahun
0,46327.46,2003
1,50779.54,2004
2,27316.89,2005


#### Case 6

Pada tahun 2003 Pemilik toko menjumpai anomali pada database, Pemilik toko mendapati masih terdapat order yang berstatus "On Hold", untuk menyelidiki hal tersebut pemilik toko ingin seluruh informasi order pada tahun 2003 yang berstatus "On Hold".

In [ ]:
# Top 3 Product Pada Tahun 2003
pd.read_sql_query("""
                  SELECT
                    *
                  FROM
                    Orders
                  WHERE
                    ORDERDATE LIKE '2003%' AND STATUS = 'On Hold'
                  """, conn)

,index,ORDER_ID,CUSTOMER_ID,PRODUCT_ID,QUANTITYORDERED,SALES,ORDERDATE,STATUS
0,0,10107,3,2,30,2871.00,2003-01-29,On Hold
1,6,10180,2,10,29,2497.77,2003-07-24,On Hold
